In [67]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import random

# 1. Definiere die Umgebung
class Environment:
    def __init__(self):
        self.prices = None
        self.t = None
        self.current_step = 0

        self.energy_charge_scheduled = None
        self.energy_discharge_schedule = None
        self.energy_realized = None

    def reset(self, prices:np.ndarray, min_charge:int, duration:int):
        self.prices = prices
        self.t = len(prices)
        self.current_step = 0

        self.energy_charge_scheduled = np.zeros((self.t))
        self.energy_discharge_schedule = np.zeros((self.t))
        self.energy_realized = np.zeros((self.t))

        return (prices, min_charge, duration, self.t), None

    def step(self, action):
        if self.current_step < self.t:
            self.energy_charge_scheduled[self.current_step] = action
            self.current_step += 1
            reward = self.reward_function()
            return reward
        else:
            return None, None

    def reward_function(self, price, action):
        # Implementiere deine Rewardfunktion hier
        consts = self.energy_charge_scheduled * (self.prices + np.repeat(0.14, self.t)) - self.energy_discharge_schedule * self.prices + (self.energy_charged_scheduled - self.energy_realized).abs * 0.8
        return consts.sum()

# 2. Definiere das neuronale Netzwerk
class DQN(nn.Module):
    def __init__(self):
        super(DQN, self).__init__()
        self.lstm = nn.LSTM(1, 32, 3, batch_first=True)

    def forward(self, x):
        print(x)
        return self.lstm(x)

# 3. Definiere den Agenten
class DQNAgent:
    def __init__(self, state_size, action_size):
        # Hyperparameter
        self.state_size = state_size
        self.action_size = action_size
        self.gamma = 0.95  # Discount rate
        self.epsilon = 1.0  # Exploration rate
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.model = DQN(state_size, action_size)
        self.criterion = nn.MSELoss()
        self.optimizer = optim.Adam(self.model.parameters(), lr=0.001)

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        else:
            with torch.no_grad():
                q_values = self.model(torch.tensor(state, dtype=torch.float32))
                return torch.argmax(q_values).item()

In [68]:
from collections import namedtuple, deque

Transition = namedtuple('Transition',
                        ('state', 'action', 'next_state', 'reward'))


class ReplayMemory(object):

    def __init__(self, capacity):
        self.memory = deque([], maxlen=capacity)

    def push(self, *args):
        """Save a transition"""
        self.memory.append(Transition(*args))

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)

In [69]:
import math

# BATCH_SIZE is the number of transitions sampled from the replay buffer
# GAMMA is the discount factor as mentioned in the previous section
# EPS_START is the starting value of epsilon
# EPS_END is the final value of epsilon
# EPS_DECAY controls the rate of exponential decay of epsilon, higher means a slower decay
# TAU is the update rate of the target network
# LR is the learning rate of the ``AdamW`` optimizer
BATCH_SIZE = 128
GAMMA = 0.99
EPS_START = 0.9
EPS_END = 0.05
EPS_DECAY = 1000
TAU = 0.005
LR = 1e-4

env = Environment()

policy_net = DQN()
target_net = DQN()
target_net.load_state_dict(policy_net.state_dict())

optimizer = optim.AdamW(policy_net.parameters(), lr=LR, amsgrad=True)
memory = ReplayMemory(10000)


steps_done = 0


def select_action(state):
    global steps_done
    sample = random.random()
    eps_threshold = EPS_END + (EPS_START - EPS_END) * \
        math.exp(-1. * steps_done / EPS_DECAY)
    steps_done += 1
    if sample > eps_threshold:
        with torch.no_grad():
            # t.max(1) will return the largest column value of each row.
            # second column on max result is index of where max element was
            # found, so we pick action with the larger expected reward.
            return policy_net(state).max(1).indices.view(1, 1)
    else:
        return torch.tensor([0], dtype=torch.long)


episode_durations = []

In [70]:
import matplotlib.pyplot as plt

def plot_durations(show_result=False):
    plt.figure(1)
    durations_t = torch.tensor(episode_durations, dtype=torch.float)
    if show_result:
        plt.title('Result')
    else:
        plt.clf()
        plt.title('Training...')
    plt.xlabel('Episode')
    plt.ylabel('Duration')
    plt.plot(durations_t.numpy())
    # Take 100 episode averages and plot them too
    if len(durations_t) >= 100:
        means = durations_t.unfold(0, 100, 1).mean(1).view(-1)
        means = torch.cat((torch.zeros(99), means))
        plt.plot(means.numpy())

    plt.pause(0.001)  # pause a bit so that plots are updated
    if True:
        if not show_result:
            display.display(plt.gcf())
            display.clear_output(wait=True)
        else:
            display.display(plt.gcf())

In [71]:
def optimize_model():
    if len(memory) < BATCH_SIZE:
        return
    transitions = memory.sample(BATCH_SIZE)
    # Transpose the batch (see https://stackoverflow.com/a/19343/3343043 for
    # detailed explanation). This converts batch-array of Transitions
    # to Transition of batch-arrays.
    batch = Transition(*zip(*transitions))

    # Compute a mask of non-final states and concatenate the batch elements
    # (a final state would've been the one after which simulation ended)
    non_final_mask = torch.tensor(tuple(map(lambda s: s is not None,
                                          batch.next_state)), dtype=torch.bool)
    non_final_next_states = torch.cat([s for s in batch.next_state
                                                if s is not None])
    state_batch = torch.cat(batch.state)
    action_batch = torch.cat(batch.action)
    reward_batch = torch.cat(batch.reward)

    # Compute Q(s_t, a) - the model computes Q(s_t), then we select the
    # columns of actions taken. These are the actions which would've been taken
    # for each batch state according to policy_net
    state_action_values = policy_net(state_batch).gather(1, action_batch)

    # Compute V(s_{t+1}) for all next states.
    # Expected values of actions for non_final_next_states are computed based
    # on the "older" target_net; selecting their best reward with max(1).values
    # This is merged based on the mask, such that we'll have either the expected
    # state value or 0 in case the state was final.
    next_state_values = torch.zeros(BATCH_SIZE)
    with torch.no_grad():
        next_state_values[non_final_mask] = target_net(non_final_next_states).max(1).values
    # Compute the expected Q values
    expected_state_action_values = (next_state_values * GAMMA) + reward_batch

    # Compute Huber loss
    criterion = nn.SmoothL1Loss()
    loss = criterion(state_action_values, expected_state_action_values.unsqueeze(1))

    # Optimize the model
    optimizer.zero_grad()
    loss.backward()
    # In-place gradient clipping
    torch.nn.utils.clip_grad_value_(policy_net.parameters(), 100)
    optimizer.step()

In [72]:
from itertools import count

num_episodes = 50

prices = [[0,1,2,3,4,5,6,8,9]]
min_charge = [4]
duration = [9]

for i_episode in range(num_episodes):
    print(f"Episode {i_episode}")
    # Initialize the environment and get its state
    state, info = env.reset(prices[i_episode], min_charge[i_episode], duration[i_episode])
    for t in count():
        action = select_action(state)
        observation, reward, terminated, truncated, _ = env.step(action.item())
        reward = torch.tensor([reward])
        done = terminated or truncated

        if terminated:
            next_state = None
        else:
            next_state = torch.tensor(observation, dtype=torch.float32).unsqueeze(0)

        # Store the transition in memory
        memory.push(state, action, next_state, reward)

        # Move to the next state
        state = next_state

        # Perform one step of the optimization (on the policy network)
        optimize_model()

        # Soft update of the target network's weights
        # θ′ ← τ θ + (1 −τ )θ′
        target_net_state_dict = target_net.state_dict()
        policy_net_state_dict = policy_net.state_dict()
        for key in policy_net_state_dict:
            target_net_state_dict[key] = policy_net_state_dict[key]*TAU + target_net_state_dict[key]*(1-TAU)
        target_net.load_state_dict(target_net_state_dict)

        if done:
            episode_durations.append(t + 1)
            plot_durations()
            break

print('Complete')
plot_durations(show_result=True)
plt.ioff()
plt.show()

Episode 0


TypeError: Environment.reward_function() missing 2 required positional arguments: 'price' and 'action'

In [80]:
import pandas as pd

def load_data():
    df = pd.read_csv('../fabian.csv')
    df['start'] = pd.to_datetime(df['start'])
    df['List_Column'] = df['prices'].apply(eval)
    return df


In [93]:
import torch.nn as nn

class CustomLoss(nn.Module):
    def __init__(self):
        super(CustomLoss, self).__init__()

    def forward(self, inputs, targets):
        loss = self.energy_charge_scheduled * (self.prices + np.repeat(0.14, self.t)) - self.energy_discharge_schedule * self.prices + (self.energy_charged_scheduled - self.energy_realized).abs * 0.8
        return loss.mean()







class SimpleLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(SimpleLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        # Initialisierung der Hidden- und Cell-States
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size)
        
        # Forward Pass durch das LSTM-Modell
        out, _ = self.lstm(x, (h0, c0))  # out: tensor der Form (batch_size, seq_length, hidden_size)
        
        # Nur den letzten Zeitschritt verwenden
        out = self.fc(out[:, -1, :])  # out: tensor der Form (batch_size, output_size)
        return out
    




# Beispiel: Modellinitialisierung
input_size = 10  # Beispiel: Größe der Eingabe
hidden_size = 20  # Beispiel: Größe der versteckten Schicht
num_layers = 2  # Beispiel: Anzahl der LSTM-Schichten
output_size = 1  # Beispiel: Größe der Ausgabe
model = SimpleLSTM(input_size, hidden_size, num_layers, output_size)



model = SimpleLSTM(input_size, hidden_size, num_layers, output_size)

# Trainings-Loop
num_epochs = 10
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn = CustomLoss()

# Inputs: 
# - preise
# - min_charge
# - duration


df = load_data()


for epoch in range(num_epochs):
    # Forward pass

    for i in range(len(df)):
        a = torch.atleast_2d(torch.tensor(df.iloc[i]['prices']))
        print(a.shape)
        outputs = model(a)
        print(outputs)
        loss = loss_fn(outputs, targets)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()


torch.Size([1, 15])


RuntimeError: For unbatched 2-D input, hx and cx should also be 2-D but got (3-D, 3-D) tensors